# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

from madminer.delphes import DelphesProcessor

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Define observables

In [4]:
# Number of particles considered
n_leptons = 1
n_photons = 2
n_jets = 1

def setup_observables(delphesprocessor):
    # ETMiss
    delphesprocessor.add_default_observables(
        n_leptons_max=n_leptons,
        n_photons_max=n_photons,
        n_jets_max=n_jets,
        include_met=True
    )
        
    # Correlations with MET
    for s1, i1 in [('l', 0), ('a', 0)]:
        delphesprocessor.add_observable(
            'deltaphi_{}{}_met'.format(s1, i1+1),
            'abs({}[{}].phi() - met.phi())'.format(s1, i1),
            required=False,
            default=0.
        )

    # Selected correlations between particles
    for s1, i1, s2, i2 in [('l', 0, 'a', 0)]:
        delphesprocessor.add_observable(
            'm_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '({}[{}] + {}[{}]).m'.format(s1, i1, s2, i2),
            required=False,
            default=0.
        )
        delphesprocessor.add_observable(
            'deltaeta_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            'abs({}[{}].eta - {}[{}].eta)'.format(s1, i1, s2, i2),
            required=False,
            default=0.
        )
        delphesprocessor.add_observable(
            'deltaphi_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            'abs({}[{}].phi() - {}[{}].phi())'.format(s1, i1, s2, i2),
            required=False,
            default=0.
        )


## Main loop

In [5]:
n_runs_per_benchmark = 18  # Number of run_cards
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5')
    
    # Load events
    for benchmark in dp.benchmark_names:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        dp.add_hepmc_sample(
            mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
            sampled_from_benchmark=benchmark
        )
        break
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2'
    )
    
    # Set up observables
    setup_observables(dp)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples()
        
    # Save
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))
    
    # Clean up
    os.remove(mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc'.format(run_str))
    os.remove(mg_process_dir + 'Events/run_{}/tag_1_pythia8_events_delphes.root'.format(run_str))
    
    break

11:33  
11:33  ------------------------------------------------------------
11:33  |                                                          |
11:33  |  MadMiner v2018.10.10                                    |
11:33  |                                                          |
11:33  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
11:33  |                                                          |
11:33  ------------------------------------------------------------
11:33  
11:33  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_1_pythia8_events.hepmc.gz
11:33  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_1_pythia8_events.hepmc.gz
11:33  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wg

OSError: Unable to create file (unable to open file: name = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5', errno = 17, error message = 'File exists', flags = 15, o_flags = a02)